In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
# Loading in the dataset
df = pd.read_csv("./df_file.csv")

X = df['Text'].tolist()
y = df['Label'].tolist()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [4]:
# Tokenize the text (fit on training data only)
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [5]:
# Pad sequences to have consistent length
max_len = 100  # You can adjust this based on your dataset
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# Convert labels to NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

In [6]:
# Build the LSTM model
embedding_dim = 50 
vocab_size = min(max_words, len(tokenizer.word_index) + 1)

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
51/51 [==============================] - 5s 94ms/step - loss: 0.1917 - accuracy: 0.9482 - val_loss: 0.7772 - val_accuracy: 0.8146
Epoch 2/20
51/51 [==============================] - 5s 97ms/step - loss: 0.1168 - accuracy: 0.9744 - val_loss: 0.4575 - val_accuracy: 0.8539
Epoch 3/20
51/51 [==============================] - 5s 95ms/step - loss: 0.1134 - accuracy: 0.9813 - val_loss: 0.8270 - val_accuracy: 0.7303
Epoch 4/20
51/51 [==============================] - 5s 95ms/step - loss: 0.2126 - accuracy: 0.9719 - val_loss: 0.5037 - val_accuracy: 0.8371
Epoch 5/20
51/51 [==============================] - 5s 94ms/step - loss: 0.0663 - accuracy: 0.9906 - val_loss: 0.5113 - val_accuracy: 0.8596
Epoch 6/20
51/51 [==============================] - 5s 95ms/step - loss: 0.0266 - accuracy: 0.9969 - val_loss: 0.5069 - val_accuracy: 0.8708
Epoch 7/20
51/51 [==============================] - 5s 94ms/step - loss: 0.0247 - accuracy: 0.9944 - val_loss: 0.8347 - val_accuracy: 0.8315
Epoch 8/20
51

In [10]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

# Make predictions on new data
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Print classification report
print(classification_report(y_test, predicted_labels))

14/14 [==============================] - 1s 36ms/step - loss: 0.6541 - accuracy: 0.8449
Test accuracy: 0.8449438214302063
14/14 [==============================] - 0s 30ms/step
              precision    recall  f1-score   support

           0       0.65      0.89      0.75        84
           1       0.92      0.91      0.92       102
           2       0.97      0.71      0.82        80
           3       0.84      0.84      0.84        77
           4       0.92      0.84      0.88       102

    accuracy                           0.84       445
   macro avg       0.86      0.84      0.84       445
weighted avg       0.87      0.84      0.85       445

